In [1]:
import time
import pymongo
import datetime
import pprint
import pandas as pd
import json

# 1 - Storage Engines (executado no Linux)

In [ ]:
mongo_mmapv1 = pymongo.MongoClient(port=28000)
mongo_wiredtiger = pymongo.MongoClient(port=29000)

In [2]:
mongo_mmapv1["benchmark"].command("serverStatus")["storageEngine"]

{'name': 'mmapv1',
 'supportsCommittedReads': False,
 'readOnly': False,
 'persistent': True}

In [3]:
mongo_wiredtiger["benchmark"].command("serverStatus")["storageEngine"]

{'name': 'wiredTiger',
 'supportsCommittedReads': True,
 'readOnly': False,
 'persistent': True}

# 2 - Import and Insert Json em uma collection

In [2]:
client = pymongo.MongoClient()

db = client['tp08']
col_clients = db['clients']

In [3]:
client.server_info()

{'version': '5.0.6',
 'gitVersion': '212a8dbb47f07427dae194a9c75baec1d81d9259',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [5, 0, 6, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': 'windows',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.29.30139 for x64',
  'ccflags': '/nologo /WX /FImongo/platform/basic.h /EHsc /W3 /wd4068 /wd4244 /wd4267 /wd4290 /wd4351 /wd4355 /wd4373 /wd4800 /wd4251 /wd4291 /we4013 /we4099 /we4930 /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /permissive- /Zc:__cplusplus /Zc:sizedDealloc /volatile:iso /diagnostics:caret /std:c++17 /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.29.30139 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',
  'target_o

In [5]:
data_json = pd.read_json(r'C:\Users\rafae\Documents\DR3_E01_LR01_clientes.json', orient='records', lines=True)
data_json.head()
#json.loads(data_jason)


,nome,telefone,endereco,cidade,uf,grauFidelidade,ativo,dependentes
0,Joaquim Maria Machado de Assis,32-2113-6009,"Rua Rio de Janeiro, 1839",Bicas,MG,87.60,1.0,NaN
1,Joao Paulo Emílio Cristovao dos Santos Coelho ...,32-2431-6112,"Rua Rio de Janeiro, 1881/201",Bicas,MG,50.00,0.0,"[{'nome': 'Maria', 'grau': 'e'}, {'nome': 'Ped..."
2,Afonso Henriques de Lima Barreto,NaN,NaN,NaN,MG,30.33,1.0,NaN
3,Julia Valentim da Silveira Lopes de Almeida,NaN,NaN,NaN,MG,7.70,1.0,NaN
4,Joao Simoes Lopes Neto,NaN,NaN,NaN,RJ,99.00,1.0,NaN


In [6]:
results = col_clients.insert_many(
    data_json.to_dict(orient='records')
)

In [4]:
client.list_database_names()

['admin',
 'airbnb',
 'benchmark',
 'config',
 'faculdade',
 'funcionarios',
 'local',
 'loja',
 'relacoes_referencia',
 'tp02',
 'tp08',
 'tree_ref_parent']

In [5]:
list(col_clients.find().limit(5))

[{'_id': ObjectId('629f53a4a98d71d99aeab23a'),
  'nome': 'Joaquim Maria Machado de Assis',
  'telefone': '32-2113-6009',
  'endereco': 'Rua Rio de Janeiro, 1839',
  'cidade': 'Bicas',
  'uf': 'MG',
  'grauFidelidade': 87.6,
  'ativo': 1.0,
  'dependentes': nan},
 {'_id': ObjectId('629f53a4a98d71d99aeab23b'),
  'nome': 'Joao Paulo Emílio Cristovao dos Santos Coelho Barreto',
  'telefone': '32-2431-6112',
  'endereco': 'Rua Rio de Janeiro, 1881/201',
  'cidade': 'Bicas',
  'uf': 'MG',
  'grauFidelidade': 50.0,
  'ativo': 0.0,
  'dependentes': [{'nome': 'Maria', 'grau': 'e'},
   {'nome': 'Pedro', 'grau': 'f'}]},
 {'_id': ObjectId('629f53a4a98d71d99aeab23c'),
  'nome': 'Afonso Henriques de Lima Barreto',
  'telefone': nan,
  'endereco': nan,
  'cidade': nan,
  'uf': 'MG',
  'grauFidelidade': 30.33,
  'ativo': 1.0,
  'dependentes': nan},
 {'_id': ObjectId('629f53a4a98d71d99aeab23d'),
  'nome': 'Julia Valentim da Silveira Lopes de Almeida',
  'telefone': nan,
  'endereco': nan,
  'cidade': n

In [10]:
#col_clients.create_index([ ('nome','uf') ])
col_clients.create_index([('nome', 1), ('uf', 1), ('cidade', 1)])

'nome_1_uf_1_cidade_1'

In [10]:
col_clients.create_index([ ('nome', pymongo.ASCENDING) ])

'nome_1'

In [12]:
col_clients.create_index([ ('uf', pymongo.ASCENDING) ])

'uf_1'

In [13]:
col_clients.create_index([ ('cidade', pymongo.ASCENDING) ])

'cidade_1'

# 3 - LOOP INSERT E CRIACAO DE INDICES

In [7]:
col_lista = db['lista']

In [16]:
for i in range(100000): 
    col_lista.insert_one({'x': i})



In [8]:
for post in col_lista.find().limit(10):
    pprint.pprint(post)

{'_id': ObjectId('629f585fa98d71d99aeab25b'), 'x': 0}
{'_id': ObjectId('629f585fa98d71d99aeab25c'), 'x': 1}
{'_id': ObjectId('629f585fa98d71d99aeab25d'), 'x': 2}
{'_id': ObjectId('629f585fa98d71d99aeab25e'), 'x': 3}
{'_id': ObjectId('629f585fa98d71d99aeab25f'), 'x': 4}
{'_id': ObjectId('629f585fa98d71d99aeab260'), 'x': 5}
{'_id': ObjectId('629f585fa98d71d99aeab261'), 'x': 6}
{'_id': ObjectId('629f585fa98d71d99aeab262'), 'x': 7}
{'_id': ObjectId('629f585fa98d71d99aeab263'), 'x': 8}
{'_id': ObjectId('629f585fa98d71d99aeab264'), 'x': 9}


In [10]:
list(col_lista.find ( { 'x' : 1000 } ))


[{'_id': ObjectId('629f585fa98d71d99aeab643'), 'x': 1000}]

In [11]:
cursor_one = col_lista.find ( { 'x' : 1000 } )

plan = cursor_one.explain()
print(json.dumps(plan, indent=4))

{
    "explainVersion": "1",
    "queryPlanner": {
        "namespace": "tp08.lista",
        "indexFilterSet": false,
        "parsedQuery": {
            "x": {
                "$eq": 1000
            }
        },
        "maxIndexedOrSolutionsReached": false,
        "maxIndexedAndSolutionsReached": false,
        "maxScansToExplodeReached": false,
        "winningPlan": {
            "stage": "COLLSCAN",
            "filter": {
                "x": {
                    "$eq": 1000
                }
            },
            "direction": "forward"
        },
        "rejectedPlans": []
    },
    "executionStats": {
        "executionSuccess": true,
        "nReturned": 1,
        "executionTimeMillis": 48,
        "totalKeysExamined": 0,
        "totalDocsExamined": 100000,
        "executionStages": {
            "stage": "COLLSCAN",
            "filter": {
                "x": {
                    "$eq": 1000
                }
            },
            "nReturned": 1,
       

# Foi usado coll_scan entao precisa-se criar indices para melhorar o plano de execucao usando indices.

# 4- Acompanhando as Insercoes

In [ ]:
ver resposta no pdf

# 5- Criando um terceiro servidor sem autenticacao e entao um usuario admin

In [2]:
client = pymongo.MongoClient(port=20000)

In [6]:
client = pymongo.MongoClient(
    username='dba',
    password='senha',port=20000
)

In [7]:
client.list_database_names()

['admin', 'config', 'local']

# 6) Crie um usuário, o primeiro, possuindo prerrogativas de administrar qualquer banco e modificar ou ler dados também em qualquer base

In [22]:
db_admin = client['admin']

db_admin.command(
    "createUser",
    "myUserAdmin2",
    pwd="raf321",
    roles=[
        {"role": "userAdminAnyDatabase", "db": "admin"},
        "readWriteAnyDatabase"
    ]
)


{'ok': 1.0}

# 7) Depois de criado o primeiro usuário, desconecte-se, habilite a autenticação e faça uma nova conexão sem autenticação. Tente criar outro usuário com máximas prerrogativas. O que ocorreu?

In [ ]:
Resposta no pdf

# 8) Crie mais dois usuários, exclusivos para os bancos de dados bd1 e bd2
9 ) Mostre os usuarios criados

In [9]:
db_admin = client['admin']

In [11]:
db_admin.command(
    "createUser",
    "User1",
    pwd="abc123",
    roles=[
        {'role': 'readWrite', 'db': 'bd1'}
    ]
)

{'ok': 1.0}

In [12]:
db_admin.command(
    "createUser",
    "User2",
    pwd="abc1234",
    roles=[
        {'role': 'readWrite', 'db': 'bd2'}
    ]
)

{'ok': 1.0}

In [16]:
list(db_admin.system.users.find())

[{'_id': 'admin.dba',
  'userId': Binary(b'\xd9\x1d\x03\x81]\nH\x99\xb9\xb2j\xbb\xcb\n\x0f\xb6', 4),
  'user': 'dba',
  'db': 'admin',
  'credentials': {'SCRAM-SHA-1': {'iterationCount': 10000,
    'salt': 'v7QdOjzHtI2oQ/uxT8nmMg==',
    'storedKey': '+x9LHCJSRCtepV+/s3nqRRajvc8=',
    'serverKey': 'r/K5eDsQaETfFhKtN7gE2vr4A2E='},
   'SCRAM-SHA-256': {'iterationCount': 15000,
    'salt': 'la7OQt5pPIQoFw+LiaX3NgSo/I0TB1R+o1OqCA==',
    'storedKey': '9peXpFeF8Wkq2drikOaqjaeqBTE0M4vd2o86IPzV4jY=',
    'serverKey': 'cHtGeYI9/pCDUpSwG8r8BQ0pGZ/ATzUelqbmVdsPudg='}},
  'roles': [{'role': 'readWriteAnyDatabase', 'db': 'admin'},
   {'role': 'userAdminAnyDatabase', 'db': 'admin'}]},
 {'_id': 'admin.User1',
  'userId': Binary(b'\xc4kdM\xe5\xd8Af\x85 \x085\xa8="_', 4),
  'user': 'User1',
  'db': 'admin',
  'credentials': {'SCRAM-SHA-1': {'iterationCount': 10000,
    'salt': 'T94qvpjQmOuivnJsqPCCjA==',
    'storedKey': 'UiJLjKaDj3N5d1W9C9IpN2eJ4fc=',
    'serverKey': 'rQ5Gg5IR1h+/ow6QGZeu6S14PhY='}

In [17]:
client.list_database_names()

['admin', 'config', 'local']